In [2]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer

In [3]:
class DataFrames:
    def __init__(
        self,
        train_file_name: str,
        test_file_name: str,
        evaluation_file_name: str,
    ) -> None:
        self.content_col_name: str = "content"
        self.content_clean_col_name: str = "content_clean"
        self.content_clean_col_name_no_UNK: str = "content_clean_no_UNK"
        self.content_clean_col_name_truc: str = "content_clean_truc"
        self.label_col_name: str = "label"
        self.train: pd.DataFrame = self.load_data(train_file_name)
        self.test: pd.DataFrame = self.load_data(test_file_name)
        self.evaluation: pd.DataFrame = self.load_data(evaluation_file_name)

    def load_data(self, file_name: str) -> pd.DataFrame:
        df = pd.read_csv(f"dataset/{file_name}.csv", sep=";")
        df.dropna(inplace=True)
        if not self.content_clean_col_name in df.columns:
            df[self.content_col_name] = df["title"] + " " + df["text"]
            df = df.drop(columns=["Unnamed: 0"])
        return df

    def get_datasets(self) -> list[pd.DataFrame]:
        return [self.train, self.test, self.evaluation]

    def get_info(self) -> str:
        test_info = self.test.shape
        train_info = self.train.shape
        evaluation_info = self.evaluation.shape
        return f"DataFrame Shapes:\n\tTrain: {train_info}\n\tTest: {test_info}\n\tEvaluation: {evaluation_info}\n"

    def save_clean(self, token_limit: int = 10000, num_words_trunc: int = 256) -> None:
        # CLEAN
        self._init_clean_content([self.train, self.test, self.evaluation])
        self.train = self.clean_df(self.train)
        self.test = self.clean_df(self.test)
        self.evaluation = self.clean_df(self.evaluation)

        most_common_words = self.get_most_common_words_counter(
            [self.test, self.train],
            token_limit,
            self.content_clean_col_name,
        )
        self.train = DataFrames.set_least_common_UNK(
            self.train, "content_clean", most_common_words
        )
        self.test = DataFrames.set_least_common_UNK(
            self.test, "content_clean", most_common_words
        )
        self.evaluation = DataFrames.set_least_common_UNK(
            self.evaluation, "content_clean", most_common_words
        )
        self.train = DataFrames.drop_least_common(
            self.train,
            self.content_clean_col_name,
            self.content_clean_col_name_no_UNK,
            most_common_words,
        )
        self.test = DataFrames.drop_least_common(
            self.test,
            self.content_clean_col_name,
            self.content_clean_col_name_no_UNK,
            most_common_words,
        )
        self.evaluation = DataFrames.drop_least_common(
            self.evaluation,
            self.content_clean_col_name,
            self.content_clean_col_name_no_UNK,
            most_common_words,
        )
        self.train = DataFrames.trunc_text(
            self.train,
            self.content_clean_col_name_no_UNK,
            self.content_clean_col_name_truc,
            num_words_trunc,
        )
        self.test = DataFrames.trunc_text(
            self.test,
            self.content_clean_col_name_no_UNK,
            self.content_clean_col_name_truc,
            num_words_trunc,
        )
        self.evaluation = DataFrames.trunc_text(
            self.evaluation,
            self.content_clean_col_name_no_UNK,
            self.content_clean_col_name_truc,
            num_words_trunc,
        )
        print(self.test.columns)
        # SAVE
        cols_to_save_clean: list[str] = [
            self.content_clean_col_name,
            self.content_clean_col_name_no_UNK,
            self.content_clean_col_name_truc,
            self.label_col_name,
        ]
        self.train.to_csv(
            f"dataset/train_clean.csv",
            sep=";",
            columns=cols_to_save_clean,
        )
        self.test.to_csv(
            f"dataset/test_clean.csv",
            sep=";",
            columns=cols_to_save_clean,
        )
        self.evaluation.to_csv(
            f"dataset/evaluation_clean.csv",
            sep=";",
            columns=cols_to_save_clean,
        )

    def num_unique_words(self, col_name: str) -> int:
        result: set = set()
        df = pd.concat([self.train, self.test, self.evaluation], ignore_index=True)
        df[col_name].str.lower().str.split().apply(result.update)
        return len(result)

    def get_vocab(self, col_name) -> dict[str, int]:
        vectorizer = CountVectorizer()
        for df in [self.train, self.test, self.evaluation]:
            vectorizer.fit_transform(df[col_name].values)
        return vectorizer.vocabulary_

    def _init_clean_content(self, dfs: list[pd.DataFrame]) -> list[pd.DataFrame]:
        for df in dfs:
            df[self.content_clean_col_name] = df[self.content_col_name]
        return dfs

    def clean_df(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self.to_lower(df, self.content_clean_col_name)
        df = self.remove_punctuation(df, self.content_clean_col_name)
        df = self.remove_stopword(df, self.content_clean_col_name)
        return df

    @staticmethod
    def to_lower(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
        df[col_name] = df[col_name].apply(lambda x: str(x).lower())
        return df

    @staticmethod
    def remove_punctuation(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
        re_punctuation = f'[{re.escape(string.punctuation)}"”“]'
        df[col_name] = df[col_name].apply(
            lambda x: re.sub(re_punctuation, " ", str(x))
            .lower()
            .replace("'s", "")
            .replace("’s", "")
        )
        return df

    @staticmethod
    def remove_stopword(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
        stop_words = set(stopwords.words("english"))
        df[col_name] = df[col_name].apply(
            lambda x: " ".join(
                word for word in str(x).split() if not word in stop_words
            )
        )
        return df

    @staticmethod
    def get_most_common_words_counter(
        dfs: list[pd.DataFrame],
        token_limit: int,
        col_name: str,
    ) -> Counter:
        word_counter: Counter = Counter()
        for df in dfs:
            if col_name not in df.columns:
                raise ValueError("Each DataFrame must have a 'clean_content' column")
            tokens = " ".join(df[col_name].astype(str)).split()
            word_counter.update(tokens)
        return Counter(dict(word_counter.most_common(token_limit)))

    @staticmethod
    def set_least_common_UNK(
        df: pd.DataFrame,
        col_name: str,
        most_common_words: Counter,
    ) -> pd.DataFrame:
        df[col_name] = df[col_name].apply(
            lambda x: " ".join(
                [
                    word if word in most_common_words else "<UNK>"
                    for word in str(x).split()
                ]
            )
        )
        return df

    @staticmethod
    def drop_least_common(
        df: pd.DataFrame,
        col_name: str,
        col_name_no_unk: str,
        most_common_words: Counter,
    ) -> pd.DataFrame:
        df[col_name_no_unk] = df[col_name].apply(
            lambda x: " ".join(
                [word for word in str(x).split() if word in most_common_words]
            )
        )
        return df

    @staticmethod
    def trunc_text(
        df: pd.DataFrame,
        col_name: str,
        col_name_trunc: str,
        trunc_num: int,
    ) -> pd.DataFrame:
        df[col_name_trunc] = df[col_name].apply(
            lambda x: " ".join(str(x).split()[:trunc_num])
        )
        return df

    @staticmethod
    def label_to_str(label: int) -> str:
        return "Fake" if label == 1 else "Not Fake"


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [16]:
class LogisticRegressionClassifier:
    def __init__(
        self,
        num_features_tfidf: int,
        ngram_range: tuple[int, int],
        logistic_regression_max_iter: int,
    ) -> None:
        self.num_features_tfidf = num_features_tfidf
        self.ngram_range = ngram_range
        self.logistic_regression_max_iter = logistic_regression_max_iter
        self.tfidf = TfidfVectorizer(
            stop_words="english",
            max_features=num_features_tfidf,
            ngram_range=ngram_range,
        )
        self.logistic_regression = LogisticRegression(
            max_iter=logistic_regression_max_iter,
        )
        self.pipeline = Pipeline(
            [
                (
                    "tfidf",
                    self.tfidf,
                ),
                (
                    "logistic-regression",
                    self.logistic_regression,
                ),
            ]
        )
        self.y_prediction = ...

    def __str__(self) -> str:
        tfidf_info: str = (
            f"TFIDF: (features: {self.num_features_tfidf}, ngram_range: {self.ngram_range})"
        )
        log_reg_info: str = (
            f"LOGISTIC-REGRESSION: (max iter: {self.logistic_regression_max_iter})"
        )
        return f"{tfidf_info}\n{log_reg_info}\n"

    def print_info(self) -> None:
        print(self)

    def train(self, X_train, y_train) -> None:
        self.pipeline.fit(X_train, y_train)

    def predict(self, X_test):
        return self.pipeline.predict(X_test)

    def print_report(self, y_test, y_hat) -> None:
        report = classification_report(y_test, y_hat, target_names=["Not Fake", "Fake"])
        print(f"{self.__class__.__name__} Report:\n{report}")


In [21]:
import time

In [29]:
def run_lgc(
    data_frames: DataFrames,
    content_col_name: str,
    label_col_name: str,
    num_features_tfidf=2000,
    ngram_range=(1, 2),
    logistic_regression_max_iter=1000,
) -> float:
    lgc = LogisticRegressionClassifier(
        num_features_tfidf,
        ngram_range,
        logistic_regression_max_iter,
    )
    start_time_lgc = time.time()
    lgc.train(
        data_frames.train[content_col_name],
        data_frames.train[label_col_name],
    )
    end_time_lgc = time.time()
    prediction = lgc.predict(data_frames.test[content_col_name])
    lgc.print_info()
    lgc.print_report(prediction, data_frames.test[label_col_name])

    rand_sample = data_frames.evaluation.sample(1)
    pred_sample = lgc.predict(rand_sample[content_col_name])
    print(rand_sample[content_col_name].values)
    label_str_true: str = data_frames.label_to_str(
        rand_sample[label_col_name].values[0]
    )
    label_str_predicted: str = data_frames.label_to_str(pred_sample[0])
    print(f"is:        {label_str_true}\npredicted: {label_str_predicted}")
    return end_time_lgc - start_time_lgc


In [23]:
# DATA
data_frames = DataFrames("train", "test", "evaluation")
data_frames_unique = data_frames.num_unique_words("content")
print(f"Unique words dataframes: {data_frames_unique}")
data_frames.save_clean(token_limit=1000)

data_frames_clean = DataFrames("train_clean", "test_clean", "evaluation_clean")
data_frames_clean_unique = data_frames_clean.num_unique_words("content_clean")
print(f"Unique words dataframes clean: {data_frames_clean_unique}")

Unique words dataframes: 394396
Index(['title', 'text', 'label', 'content', 'content_clean',
       'content_clean_no_UNK', 'content_clean_truc'],
      dtype='object')
Unique words dataframes clean: 1001


In [24]:
# TESTS WITH TIME
times: dict[str, float] = {}

In [25]:
lgc_time: float = run_lgc(
    data_frames,
    data_frames.content_col_name,
    data_frames_clean.label_col_name
)
times["LGC"] = lgc_time

TFIDF: (features: 2000, ngram_range: (1, 2))
LOGISTIC-REGRESSION: (max iter: 1000)

LogisticRegressionClassifier Report:
              precision    recall  f1-score   support

    Not Fake       0.97      0.97      0.97      3751
        Fake       0.97      0.97      0.97      4366

    accuracy                           0.97      8117
   macro avg       0.97      0.97      0.97      8117
weighted avg       0.97      0.97      0.97      8117

['Hypothetically speaking, U.S. Admiral says ready for nuclear strike on China if Trump so ordered MELBOURNE (Reuters) - The U.S. Pacific Fleet commander, addressing a security conference in Australia, said in answer to a question on Thursday that he would be prepared to launch a nuclear strike on China if President Donald Trump so ordered. The fleet spokesman later said the question was asked as an “outrageous hypothetical”. Admiral Scott Swift was speaking at the Australian National University in Canberra when he was asked whether he would be p

In [26]:
lgc_time_clean: float = run_lgc(
    data_frames_clean,
    data_frames_clean.content_clean_col_name_truc,
    data_frames_clean.label_col_name,
)
times["LGC Clean"] = lgc_time_clean

TFIDF: (features: 2000, ngram_range: (1, 2))
LOGISTIC-REGRESSION: (max iter: 1000)

LogisticRegressionClassifier Report:
              precision    recall  f1-score   support

    Not Fake       0.97      0.97      0.97      3754
        Fake       0.97      0.97      0.97      4359

    accuracy                           0.97      8113
   macro avg       0.97      0.97      0.97      8113
weighted avg       0.97      0.97      0.97      8113

['watch republicans hard cnn pass house republicans seven years come plan care act plan bad even gop majority could pass voting care act times years order conservative base republicans actually american healthcare act would health care 20 million americans many republican voters reason bill would also cause healthcare especially senior citizens would pay times much right care act vote republican bill thursday republican leadership could enough members party get number votes vote republicans control house even bring bill vote simply put donald tru

In [27]:
for fname, ftime in times.items():
    print(f"{fname}: {ftime:.5f} seconds")

LGC: 12.72050 seconds
LGC Clean: 3.03294 seconds
